In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd
import re
# Parsear el contenido HTML




In [8]:

def scrape_multiple_pages(num_iterations):
    # Inicializar el controlador de Selenium (en este ejemplo, uso Chrome, puedes ajustarlo según tu navegador)
    driver = webdriver.Chrome()
    #num_iterations=num_iterations+10
    # Lista para almacenar todos los números únicos encontrados
    all_unique_digits = []

    # Iterar sobre el número de páginas especificado
    for page_number in range(1, num_iterations + 1):
        # Construir la URL de la página actual
        url = f'https://baloto.com/resultados?page={page_number}'
        
        # Navegar a la página
        driver.get(url)
        
        # Obtener el HTML de la página actual
        html = driver.page_source
        
        # Analizar el contenido HTML con BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Lista para almacenar los resultados de la página actual
        digits = []

        # Encontrar todas las etiquetas <td>
        td_elements = soup.find_all('td')

        for td in td_elements:
            number = td.get_text().strip()  # Extraer el texto del <td>
            if re.match(r'^\d{4}$', number):  # Verificar si el texto contiene exactamente 4 dígitos
                digits.append(number)  # Añadir el número a la lista

        # Convertir la lista a un conjunto para eliminar duplicados y luego volver a convertirla en lista
        unique_digits = list(set(digits))

        # Extender la lista acumulativa con los números únicos de esta página
        all_unique_digits.extend(unique_digits)
        
        # Imprimir los números únicos de la página actual
        print(f'Página {page_number}: {unique_digits}')

    # Cerrar el navegador al finalizar
    driver.quit()

    # Devolver todos los números únicos encontrados en todas las páginas
    return all_unique_digits

# Ejemplo de uso: obtener números únicos de las primeras 3 páginas
results = scrape_multiple_pages(2)
print("Todos los números únicos encontrados:")
print(results)

Página 1: []
Página 2: []
Todos los números únicos encontrados:
[]


In [3]:
len(results)

0

## combinacion

In [4]:
df_horario = pd.DataFrame(columns=['sorteo', 'fecha_sorteo'])
df_yellow = pd.DataFrame(columns=['sorteo', 'yellows'])
df_red = pd.DataFrame(columns=['sorteo', 'reds'])

for result in results:
    url = f'https://baloto.com/resultados-baloto/{result}'
    driver = webdriver.Chrome()
    # Navegar a la página
    driver.get(url)
    # Esperar hasta que el elemento de interés esté presente en el DOM (por ejemplo, el primer elemento con la clase 'yellow-ball')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'yellow-ball')))
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'red-ball')))
    # Obtener el HTML de la página actual
    html = driver.page_source
  
    # Analizar el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    fecha_element = soup.find('div', class_='gotham-medium dark-blue')
    fecha_element = fecha_element.get_text().strip()
    
    df_temp = pd.DataFrame({'sorteo': [result],'fecha_sorteo':[fecha_element]})
    df_horario = pd.concat([df_horario, df_temp], ignore_index=True)
    
    numbers1 = []
    elements1 = soup.find_all(class_='yellow-ball')

    for element1 in elements1:
        number1 = element1.get_text().strip()  # Obtener el texto y eliminar espacios en blanco
        numbers1.append(int(number1))  # Convertir el texto a entero y añadir a la lista


    df_temp1 = pd.DataFrame({'sorteo': [result],'yellows': [numbers1]})
    df_yellow = pd.concat([df_yellow, df_temp1], ignore_index=True)
    
    reds = []
    elements_red = soup.find_all(class_='red-ball')

    for element_red in elements_red:
        red = element_red.get_text().strip()  # Obtener el texto y eliminar espacios en blanco
        reds.append(int(red))  # Convertir el texto a entero y añadir a la lista

    df_temp2 = pd.DataFrame({'sorteo': [result],'reds': [red]})
    df_red = pd.concat([df_red, df_temp2], ignore_index=True)

In [5]:
df_combined = df_horario.merge(df_yellow, on='sorteo').merge(df_red, on='sorteo')

In [6]:
df_combined

,sorteo,fecha_sorteo,yellows,reds


In [7]:
excel_filename = './outs/resultados_baloto2.xlsx'
df_combined.to_excel(excel_filename, index=False)